# MusicVAE를 이용하여 4마디 드럼 데이터 생성하기

#### 내용 정리 노션

링크 https://www.notion.so/ethicsense/Train-Generate-MusicVAE-06867f19397a44dab614a47b8502724e

#### 코드 코랩 (생성 사운드 재생)

링크 https://colab.research.google.com/drive/11UcFIheDM_An-yHkxNsV9aUQLs9d6QAv?usp=sharing

## Set up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import glob

BASE_DIR = "gs://download.magenta.tensorflow.org/models/music_vae/colab2"

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


print('Importing libraries and defining some helper functions...')
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  print('Start Seq Reconstruction')
  play(note_sequences[0])
  print('End Seq Reconstruction')
  play(note_sequences[-1])
  print('Mean Sequence')
  play(note_sequences[num_steps // 2])
  print('Start -> End Interpolation')
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

print('Done')

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 155639 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 1.4 MB 34.4 MB/s 
     |████████████████████████████████| 90 kB 10.7 MB/s 
     |████████████████████████████████| 1.4 MB 68.4 MB/s 
     |████████████████████████████████| 5.6 MB 13.6 MB/s 
     |████████████████████████████████| 204 kB 62.1 MB/s 
     |██████████████

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


Instructions for updating:
non-resource variables are not supported in the long term
Done


## Data Load

In [ ]:
import tensorflow_datasets as tfds

# tfds works in both Eager and Graph modes
tf.enable_eager_execution()

# Load the full GMD with MIDI only (no audio) as a tf.data.Dataset
dataset, info = tfds.load(
    name="groove/4bar-midionly",
    split=tfds.Split.TRAIN,
    with_info=True,
    try_gcs=True)

In [ ]:
info

tfds.core.DatasetInfo(
    name='groove',
    version=2.0.1,
    description='The Groove MIDI Dataset (GMD) is composed of 13.6 hours of aligned MIDI and
(synthesized) audio of human-performed, tempo-aligned expressive drumming
captured on a Roland TD-11 V-Drum electronic drum kit.',
    homepage='https://g.co/magenta/groove-dataset',
    features=FeaturesDict({
        'bpm': tf.int32,
        'drummer': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
        'id': tf.string,
        'midi': tf.string,
        'style': FeaturesDict({
            'primary': ClassLabel(shape=(), dtype=tf.int64, num_classes=18),
            'secondary': tf.string,
        }),
        'time_signature': ClassLabel(shape=(), dtype=tf.int64, num_classes=5),
        'type': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    }),
    total_num_examples=21415,
    splits={
        'test': 2033,
        'train': 17261,
        'validation': 2121,
    },
    supervised_keys=None,
    citation="""@inpro

In [ ]:
# Build your input pipeline
dataset = dataset.shuffle(1024).batch(32).prefetch(
    tf.data.experimental.AUTOTUNE)
for features in dataset.take(1):
  # Access the features you are interested in
  midi, genre = features["midi"], features["style"]["primary"]

데이터를 코드로 직접 불러오고 코드블럭 내에서만 조작하고자 했으나 tensorflow_datasets 모듈에 대한 이해가 부족하고 적절한 자료를 찾지 못해 시간이 너무 많이 지연되어 기한을 맞추고자 우선 직접 다운로드 받아 드라이브에 저장하고 Magenta에서 지원하는 스크립트를 사용하여 전처리를 수행하기로 했습니다.

### Data load 2nd try

In [ ]:
!convert_dir_to_note_sequences \
  --input_dir='./drive/MyDrive/VAE/datasets/midionly' \
  --output_file='./drive/MyDrive/VAE/datasets/sequences.tfrecord' \
  --recursive